Jupyter notebook for analyzing how concatenated v. RMG built mechanisms compare

In [1]:
import os 
import re
import matplotlib.pyplot as plt
import rmgpy.chemkin
import numpy as np
import cantera as ct
import random
import pandas as pd

In [2]:
def load_chemkin_file_RMG(blend_folder): 
    """ 
    Load Chemkin file and extract the reactions and species. Must provide blend folder name.
    """
    
    full_path = os.path.join('/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts_species_in_core',blend_folder,'chemkin')
    chemkin_path = os.path.join(full_path,'copies', 'copy_chem_annotated.inp')
   # chemkin_path = os.path.join(full_path,'chem_annotated.inp')
    dictionary_path = os.path.join(full_path,'species_dictionary.txt')
    transport_path = os.path.join(full_path,'tran.dat')


    species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_path, dictionary_path=dictionary_path, transport_path=transport_path)

    return species_list, reaction_list

In [3]:
def load_chemkin_file_concatenated(concatenated_folder): 
    """ 
    Load Chemkin file and extract the reactions and species.
    Must provide concatenated folder name. 
    """
    full_path = f'/work/westgroup/nora/Code/projects/halogens/refrigerants/blends/concatenate_equal_parts/{concatenated_folder}/concatenated/'
    chemkin_path = os.path.join(full_path,'copies', 'copy_chem.inp')
   # chemkin_path = os.path.join(full_path,'chem_annotated.inp')
    dictionary_path = os.path.join(full_path,'species_dictionary.txt')
    transport_path = os.path.join(full_path,'tran.dat')


    species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_path, dictionary_path=dictionary_path, transport_path=transport_path)

    return species_list, reaction_list

In [16]:
def compare_rxns(model_1_reactions, model_2_reactions): 
    '''
    Compares the reaction equations of each model. Uses to_cantera() on RMG model to eliminate chemkin_identifier 
    '''

################### mostly taken from diffmodel.py, compare_model_reactions() ###################

    #remove reactions with unknown species
    to_remove = []
    for reactionList in (model_1_reactions, model_2_reactions):
        for reaction in reactionList:
            for side in (reaction.products, reaction.reactants):
                for species in side:
                    if not species.molecule:
                        to_remove.append((reactionList, reaction))
                        print("Removing reaction {!r} that had unidentified species {!r}".format(reaction, species))
                        break
    if len(to_remove)!=0:
        for reactionList, reaction in to_remove:
            reactionList.remove(reaction)
        print(f'Following reactions removed because they have unknown species: {to_remove}')
    
    #find the common reactions and the unique reactions
    common_reactions = []
    common_reactions_equations = []
    unique_reactions_1 = []
    unique_reactions_2 = []
    copy_of_model_1_reactions = model_1_reactions.copy() #list 1
    copy_of_model_2_reactions = model_2_reactions.copy() #list 2

    
    for model_2_rxn in copy_of_model_2_reactions:
        for model_1_rxn in copy_of_model_1_reactions: # make a copy so you don't remove from the list you are iterating over
            counter = 0 #use this to see how many times this loop below was executed
            if model_2_rxn.is_isomorphic(model_1_rxn):
                common_reactions.append([model_1_rxn, model_2_rxn])
                common_reactions_equations.append([str(model_1_rxn), str(model_2_rxn)])
                copy_of_model_1_reactions.remove(model_1_rxn)
                counter += 1
            if counter>1:
                print([str(model_1_rxn),str(model_2_rxn)]) #let's see if a rxn has two matches
                
 
    #find the unique reactions of each model
    for model_2_rxn in copy_of_model_2_reactions:
        for r1, r2 in common_reactions:
            if model_2_rxn is r2:
            #if model_2_rxn.is_isomorphic(r2):
                break
        else:
            unique_reactions_2.append(model_2_rxn)
    for model_1_rxn in copy_of_model_1_reactions:
        for r1, r2 in common_reactions:
            if model_1_rxn is r1:
#             if model_1_rxn.is_isomorphic(r1):
                break
        else:
            unique_reactions_1.append(model_1_rxn)
            
    
    #find the reactions that have different kinetics (aren't identical and aren't similar)
    
    different_kinetics_reactions = []
    identical_kinetics_reactions = []
    similar_kinetics_reactions = []
    
    for r1, r2 in common_reactions: 
        if (r2.kinetics.is_identical_to(r1.kinetics)==False) and (r2.kinetics.is_similar_to(r1.kinetics)==False):
            different_kinetics_reactions.append((r1, r2))
        if r2.kinetics.is_identical_to(r1.kinetics)==True:
            identical_kinetics_reactions.append((r1, r2))
        if r2.kinetics.is_similar_to(r1.kinetics)==True:
            similar_kinetics_reactions.append((r1, r2))
            
            
    return similar_kinetics_reactions, identical_kinetics_reactions, different_kinetics_reactions, common_reactions, common_reactions_equations, unique_reactions_1, unique_reactions_2
    
    

In [40]:
def compare_species(model_1_species_list, model_2_species_list):
    """ Compares species list between models. """
    
    #two lists of same species, just have different names in each model
    common_species_1 = []
    common_species_2 = []
    
    #combined list of common species, with tuples of the two different names
    common_species = []
    
    for spc_1 in model_1_species_list:
        for spc_2 in model_2_species_list:
            if spc_1.is_isomorphic(spc_2):
                common_species_1.append(spc_1)
                common_species_2.append(spc_2)
                common_species.append((spc_1, spc_2))
                
                
    unique_species_1 = [spc_1 for spc_1 in model_1_species_list if spc_1 not in common_species_1]
    unique_species_2 = [spc_2 for spc_2 in model_2_species_list if spc_2 not in common_species_2]

                
    return common_species, unique_species_1, unique_species_2
    

In [48]:
#model 1 = concatenated, model 2 = rmgbuilt

RMG_species_list, RMG_reaction_list = load_chemkin_file_RMG('C2H5F_CH2FCH2F')
con_species_list, con_reaction_list = load_chemkin_file_concatenated('C2H5F_CH2FCH2F')

In [49]:
common_species, unique_species_conc, unique_species_rmg = compare_species(con_species_list, RMG_species_list)

In [50]:

similar_kinetics_reactions, identical_kinetics_reactions, different_kinetics_reactions, common_reactions, common_reactions_equations, unique_reactions_conc, unique_reactions_rmg = compare_rxns(con_reaction_list, RMG_reaction_list) 

In [51]:
################ #ANALYSIS OF MODELS

print(f"RMG mechanism has {len(RMG_reaction_list)} reactions and {len(RMG_species_list)} species.")
print('\n')
print(f"Concatenated mechanism reaction has {len(con_reaction_list)} reactions and {len(con_species_list)} species.")
print('\n')
print(f"common reactions: {len(common_reactions)}")
print('\n')
print(f'\tOf common reactions, {len(different_kinetics_reactions)} reactions have different kinetics')
print('\n')
print(f'\tOf common reactions, {len(identical_kinetics_reactions)} reactions have identical kinetics')
print('\n')
print(f'\tOf common reactions, {len(similar_kinetics_reactions)-len(identical_kinetics_reactions)} reactions have similar kinetics')
print('\n')
print(f'common species: {len(common_species)}')
print('\n')
print(f'unique species in Conc. model: {len(unique_species_conc)}')
print('\n')
print(f'unique species in RMG model: {len(unique_species_rmg)}')


#NOTE: concatenated models can be greatly reduced in RMG bc you get the same flamespeed with much less reactions.
# Also include data about the original mechanisms and how they compare to conc v. rmg built. 

RMG mechanism has 4725 reactions and 255 species.


Concatenated mechanism reaction has 8013 reactions and 246 species.


common reactions: 3545


	Of common reactions, 544 reactions have different kinetics


	Of common reactions, 2131 reactions have identical kinetics


	Of common reactions, 870 reactions have similar kinetics


common species: 228


unique species in Conc. model: 18


unique species in RMG model: 27


In [46]:
for i in unique_species_conc:
    print(i)

C[C]F(55)
[CH]C(56)
[CH]CF(63)
F[C]CF(66)
[CH]C(F)F-2(93)
C=C[C]F-2(110)
FC1[C]C1(113)
[C]=CC-2(117)
[C]=CCF-2(132)
C#C[C]C(142)
[C]=CC=C-2(143)
C#C[C]C=C(153)
[C]=CC#CC-2(169)
C#CC1[C]C1(174)
[C]=CC=C=C(177)


In [45]:
for i in unique_species_rmg:
    print(i)

H2CC(21)
CHF(33)
C[C]F(54)
[CH]C(55)
[CH]CF(62)
F[C]CF(65)
[CH]C(F)F-2(92)
[C]=CF-2(100)
C=C[C]F-2(109)
FC1[C]C1(112)
[C]=CC-2(116)
[C]=CCF-2(131)
C#C[C]C(141)
[C]=CC=C-2(142)
[C]=CC#C(155)
[C]=CC#CC-2(167)
C#CC1[C]C1(172)
[C]=CC=C=C(175)
CF2(178)
F[C]C(F)F(202)
Ar(218)
He(219)
C(T)(220)
CH3OH(223)
CH2CHO(224)
CH3CHO(225)
CF2CO(234)
O=[C]CF(296)
[O]OCF(320)
[O]CF(327)
[O]C(F)C(F)F(394)
CC(F)(F)F(600)
OO[C](F)F(804)
FC1(F)OC1(F)F(1561)
